In [1]:
using Pkg
Pkg.activate("../STLCG.jl/.")

 Activating environment at `~/repos/AutonomousRiskFramework/STLCG.jl/Project.toml`


In [2]:
using Revise
using STLCG
using EllipsisNotation


┌ Info: Precompiling STLCG [6ababdf7-4701-4f28-b3b2-5630f64e3c98]
└ @ Base loading.jl:1278


In [3]:
bs = 2
t = 15
x_dim = 3
dim = 2
x = randn(bs, t, x_dim);
y = randn(bs, t, x_dim);

xs = [x[:,i:i,..] for i in 1:t];


op = Always(subformula=x, interval=[5, Inf])
h0 = STLCG.init_rnn_cell!(op, x)
scale = 0
distributed = true

# rnn_cell(op, xs[1], op.h0; scale, distributed)
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[1]), xs[1]) # <---- doesn't work
# STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

true

In [13]:
op = STLCG.Negation(STLCG.LessThan(:x, 4.0))
op1 = STLCG.LessThan(:x, 0.0)
op2 = STLCG.GreaterThan(:y, 0.0)
op = STLCG.Or(subformula1=op1, subformula2=op2)
op = STLCG.Always(subformula=op, interval=nothing)

Always
  subformula: STLCG.Or
  interval: Nothing nothing
  _interval: Array{Float64}((2,)) [0.0, Inf]
  rnn_dim: Int8 1
  steps: Int8 1
  operation: Minish (function of type typeof(Minish))
  M: Array{Float64}((1, 1)) [0.0]
  b: Array{Float64}((1,)) [1.0]


In [25]:
op.subformula((x,x))

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.014236  0.0643839  0.129185  0.973042  …  0.789883  0.0558895  0.34267
 1.27809   0.366613   0.344831  1.29821      1.46592   1.78059    0.673362

[:, :, 2] =
 0.17119   0.312389  1.17013    0.310833  …  1.82768  1.03153   2.47358
 0.641186  0.692986  0.0428127  0.440219     2.10405  0.639417  0.710165

[:, :, 3] =
 0.571426  0.622713  1.35888   0.653964  …  0.161919  0.0168242  1.2357
 0.439976  0.485175  0.578168  0.252686     0.537099  1.09756    0.510397

In [19]:
outputs, states = STLCG.run_rnn_cell(op, (x,x); scale=0, distributed=false);

In [48]:
op((x,x); distributed=true)

LoadError: MethodError: no method matching *(::Array{Float64,3}, ::Array{Float64,3})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:538
  *(!Matched::ChainRulesCore.DoesNotExist, ::Any) at /Users/karenleung/.julia/packages/ChainRulesCore/KvORk/src/differential_arithmetic.jl:28
  *(!Matched::ChainRulesCore.One, ::Any) at /Users/karenleung/.julia/packages/ChainRulesCore/KvORk/src/differential_arithmetic.jl:98
  ...

In [58]:
xx = STLCG.robustness_trace(op.subformula, (x,y))
hc = STLCG.init_rnn_cell!(op, xx)
time_dim = size(xx)[2]

15

In [66]:
h0, c = hc
z = xx[:,1:1,..]
new_h = (h0 * c + z) .* (z .== h0) / (c + 1) + z .* (z .< h0) + h0 .* (z .> h0)
new_c = (c + 1.0) .* (z .== h0) + 1.0 .* (z .< h0) + c .* (z .> h0)
state = (new_h, new_c)
output = new_h
o, hc = output, state

LoadError: MethodError: no method matching *(::Array{Float64,3}, ::Array{Float64,3})
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:538
  *(!Matched::ChainRulesCore.DoesNotExist, ::Any) at /Users/karenleung/.julia/packages/ChainRulesCore/KvORk/src/differential_arithmetic.jl:28
  *(!Matched::ChainRulesCore.One, ::Any) at /Users/karenleung/.julia/packages/ChainRulesCore/KvORk/src/differential_arithmetic.jl:98
  ...

In [95]:
op((x,x); distributed=true, scale=0)

2×1×3 Array{Float64,3}:
[:, :, 1] =
 0.0030889667542218216
 0.021055287814701746

[:, :, 2] =
 0.17093208932121723
 0.04281270670745427

[:, :, 3] =
 0.016824175231990146
 0.0703969655196175

In [96]:
vcat(STLCG.gradient(x -> sum(op((x,x); distributed=true)), x)...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  -1.0  0.0  0.0  0.0

In [99]:
vcat(STLCG.gradient((x, y) -> sum(STLCG.robustness_trace(op, (x,y); distributed=true)), x,y)...)[:,:,1]'

15×4 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  1.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  1.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0
 -0.0  -0.0  0.0  0.0

In [9]:
vcat(STLCG.gradient(x -> sum(STLCG.run_rnn_cell(op, x, x, scale=0)[1][:,end,:]), x)...)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3] =
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [103]:
□ϕ = STLCG.□(op, interval=[2,4])

Always
  subformula: Always
  interval: Array{Int64}((2,)) [2, 4]
  _interval: Array{Int64}((2,)) [2, 4]
  rnn_dim: Int8 4
  steps: Int8 5
  operation: Minish (function of type typeof(Minish))
  M: Array{Float64}((4, 4)) [0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0; 0.0 0.0 0.0 0.0]
  b: Array{Float64}((4,)) [0.0, 0.0, 0.0, 1.0]


In [141]:
robustness_trace(□ϕ, x; args...)

LoadError: MethodError: no method matching robustness_trace(::Always, ::Array{Float64,3}; keepdims=true, dims=4, distributed=false, scale=0, pscale=1)
Closest candidates are:
  robustness_trace(!Matched::STLCG.Or, ::Any; dims, pscale, scale, keepdims, distributed) at In[54]:44
  robustness_trace(!Matched::STLCG.LessThan, ::Any; pscale, kwargs...) at In[54]:2
  robustness_trace(!Matched::STLCG.Implies, ::Any; pscale, scale, keepdims, distributed) at In[54]:7 got unsupported keyword argument "dims"
  ...

In [ ]:
STLCG.gradient(x -> sum(STLCG.rnn_cell(op, x, h0; scale, distributed)[2][1][2]), xs[1]) # <--- works

In [4]:
ϕ = STLCG.LessThan(:x, 5)
ψ = STLCG.GreaterThan(:x, -3)
op = STLCG.Or(subformula1=STLCG.And(subformula1=ϕ, subformula2=ψ), subformula2=ϕ)

STLCG.Or
  subformula1: STLCG.And
  subformula2: STLCG.LessThan
  operation: Maxish (function of type typeof(Maxish))


In [191]:
(f::STLCG.Formula)(x::Array{Float64}; kwargs...) = robustness_trace(f, x; kwargs...)

In [192]:
ϕ(x)

2×15×3 Array{Float64,3}:
[:, :, 1] =
 4.33878  4.55352  4.0096   4.28465  …  4.87325  4.24155  4.18938  4.36833
 4.35477  4.20984  4.53724  4.8834      4.25997  4.16101  4.62856  4.13871

[:, :, 2] =
 4.19292  4.05989  4.26052  4.3984   …  4.02068  4.74747  4.05824  4.06667
 4.13728  4.53258  4.77843  4.72391     4.18943  4.02867  4.20342  4.66529

[:, :, 3] =
 4.47263  4.89431  4.49562  4.28735  …  4.9606   4.3826   4.59063  4.70413
 4.29403  4.08497  4.01783  4.86847     4.10466  4.29218  4.78882  4.30469

In [147]:
struct Predicate
    x::Symbol
end

In [157]:
(p::Primitive)(x::Int) = (2*x, p.x)
(p::Primitive)(x::Real) = 3*x

In [158]:
a = Primitive(:asadsadsa)

Primitive(:asadsadsa)

In [159]:
a(1), a(1.)

((2, :asadsadsa), 3.0)

In [148]:
Primitive(:x) < 5

Primitive(:x)

In [172]:
Primitive(:a) > Primitive(:b)

false

In [170]:
(Base.:(<))(a::Symbol, b::Primitive) = a.x < b.x

In [75]:
function <(a,b)
    a < b
end

< (generic function with 1 method)

In [78]:
< |> methods

LoadError: syntax: "<" is not a unary operator